# JPEG vs YUV420 Visual Verification

Head-to-head comparison of the JPEG and YUV420 image formats in `SlipstreamLoader`.

YUV420 stores decoded images as raw planar YUV 4:2:0 (chroma subsampled), eliminating
JPEG Huffman decode + IDCT at load time. The round-trip (JPEG → RGB → YUV420P → RGB)
introduces small pixel-level errors from chroma subsampling. This notebook verifies
the outputs are visually identical and quantifies the pixel differences.

In [ ]:
LITDATA_VAL_PATH = "s3://visionlab-datasets/imagenet1k/pre-processed/s256-l512-jpgbytes-q100-streaming/val/"

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from slipstream import SlipstreamDataset, SlipstreamLoader, CenterCrop

dataset = SlipstreamDataset(
    remote_dir=LITDATA_VAL_PATH,
    decode_images=False,
)
print(f"Dataset: {len(dataset):,} samples")

## Load the same batch with JPEG and YUV420

We use `CenterCrop(224)` with `shuffle=False` so both formats decode the exact same
images with the same deterministic crop.

In [ ]:
def load_batch(image_format):
    loader = SlipstreamLoader(
        dataset,
        batch_size=16,
        shuffle=False,
        image_format=image_format,
        pipelines={'image': [CenterCrop(224)]},
        exclude_fields=['path'],
        verbose=False,
    )
    batch = next(iter(loader))
    loader.shutdown()
    return batch

batch_jpeg = load_batch("jpeg")
batch_yuv = load_batch("yuv420")

img_jpeg = batch_jpeg['image']  # [B, 3, 224, 224] uint8
img_yuv = batch_yuv['image']
print(f"JPEG:   {img_jpeg.shape}, dtype={img_jpeg.dtype}")
print(f"YUV420: {img_yuv.shape}, dtype={img_yuv.dtype}")

## Side-by-side comparison

Top row: JPEG decode. Bottom row: YUV420 decode. Should look identical.

In [ ]:
n = 8
fig, axes = plt.subplots(2, n, figsize=(n * 1.8, 3.8))
for i in range(n):
    axes[0][i].imshow(img_jpeg[i].permute(1, 2, 0).numpy())
    axes[0][i].axis('off')
    axes[1][i].imshow(img_yuv[i].permute(1, 2, 0).numpy())
    axes[1][i].axis('off')
axes[0][0].set_ylabel('JPEG', fontsize=11)
axes[1][0].set_ylabel('YUV420', fontsize=11)
fig.suptitle('CenterCrop(224): JPEG vs YUV420', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Pixel-level difference analysis

The YUV420 round-trip (RGB → YUV420P → RGB) introduces small errors from:
1. **Chroma subsampling** — U/V planes are half-resolution (2x2 averaging)
2. **Fixed-point BT.601 conversion** — integer rounding in both directions

We expect most pixels to differ by 0-2, with occasional outliers up to ~5.

In [ ]:
diff = (img_jpeg.float() - img_yuv.float()).abs()

print(f"Pixel difference statistics (across entire batch):")
print(f"  Mean absolute error:  {diff.mean():.3f}")
print(f"  Max absolute error:   {diff.max():.0f}")
print(f"  Median:               {diff.median():.0f}")
print(f"  % pixels with diff=0: {(diff == 0).float().mean() * 100:.1f}%")
print(f"  % pixels with diff≤1: {(diff <= 1).float().mean() * 100:.1f}%")
print(f"  % pixels with diff≤2: {(diff <= 2).float().mean() * 100:.1f}%")
print(f"  % pixels with diff>5: {(diff > 5).float().mean() * 100:.2f}%")

In [ ]:
# Show difference heatmap for first 4 images
fig, axes = plt.subplots(3, 4, figsize=(9, 7))
for i in range(4):
    axes[0][i].imshow(img_jpeg[i].permute(1, 2, 0).numpy())
    axes[0][i].set_title(f'JPEG #{i}', fontsize=9)
    axes[0][i].axis('off')

    axes[1][i].imshow(img_yuv[i].permute(1, 2, 0).numpy())
    axes[1][i].set_title(f'YUV420 #{i}', fontsize=9)
    axes[1][i].axis('off')

    # Per-pixel max diff across channels
    pixel_diff = diff[i].max(dim=0).values.numpy()
    im = axes[2][i].imshow(pixel_diff, cmap='hot', vmin=0, vmax=5)
    axes[2][i].set_title(f'|diff| (max={pixel_diff.max():.0f})', fontsize=9)
    axes[2][i].axis('off')

fig.colorbar(im, ax=axes[2], shrink=0.8, label='Pixel difference')
fig.suptitle('Pixel-level difference: JPEG vs YUV420', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Histogram of per-pixel differences
fig, ax = plt.subplots(figsize=(7, 3.5))
values = diff.flatten().numpy()
max_val = int(values.max())
counts, bins, _ = ax.hist(values, bins=np.arange(-0.5, max_val + 1.5, 1),
                          edgecolor='black', linewidth=0.5)
ax.set_xlabel('Absolute pixel difference')
ax.set_ylabel('Count')
ax.set_title('Distribution of per-pixel errors (JPEG vs YUV420)', fontweight='bold')
ax.set_yscale('log')
ax.set_xlim(-0.5, max_val + 0.5)
plt.tight_layout()
plt.show()